In [169]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords


In [170]:
movies_data = pd.read_csv('IMDBDataset.csv')
movies_data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [171]:
movies_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50000 non-null  object
 1   sentiment  50000 non-null  object
dtypes: object(2)
memory usage: 781.4+ KB


In [172]:
movies_data.describe()

,review,sentiment
count,50000,50000
unique,49582,2
top,Loved today's show!!! It was a variety and not...,positive
freq,5,25000


In [173]:
movies_data['review'].nunique()

49582

In [174]:
movies_data["review"].value_counts()

review
Loved today's show!!! It was a variety and not solely cooking (which would have been great too). Very stimulating and captivating, always keeping the viewer peeking around the corner to see what was coming up next. She is as down to earth and as personable as you get, like one of us which made the show all the more enjoyable. Special guests, who are friends as well made for a nice surprise too. Loved the 'first' theme and that the audience was invited to play along too. I must admit I was shocked to see her come in under her time limits on a few things, but she did it and by golly I'll be writing those recipes down. Saving time in the kitchen means more time with family. Those who haven't tuned in yet, find out what channel and the time, I assure you that you won't be disappointed.                                                                                                                                                                                                         

In [175]:
movies_data.drop_duplicates(subset=['review'],inplace=True)

In [176]:
# remove HTML tags from the string
import re

def remove_html_tags(text):
    clean = re.sub(r'<[^>]+>', '', text)
    return clean

movies_data['review']= movies_data['review'].apply(lambda msg: remove_html_tags(msg))


In [177]:

movies_data['sentiment'] = movies_data['sentiment'].map({'positive': 1, 'negative': 0})
movies_data.head(4)

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. The filming tec...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0


In [ ]:
movies_data['Tokenized'] = [[word for word in document.lower().split() if word not in stopwords.words('english')]
         for document in movies_data['review']]

In [ ]:
X=movies_data['Tokenized']
y= movies_data['sentiment']
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42)



In [ ]:
from gensim.models import Word2Vec

# Train Word2Vec on X_train only
word2vec_model = Word2Vec(X_train, vector_size=200, window=5, min_count=1, workers=4)

word2vec_model.build_vocab(X_train, update=True)
word2vec_model.train(X_train, total_examples=word2vec_model.corpus_count, epochs=word2vec_model.epochs)

In [ ]:

def get_average_word_vector(words, model):
    vectors = [model.wv[word] for word in words if word in model.wv]
    if len(vectors) > 0:
        return np.mean(vectors, axis=0)
    else:
        return np.zeros(model.vector_size)

In [ ]:

# Transform X_train and X_test into feature vectors
X_train_vectors = np.array([get_average_word_vector(words, word2vec_model) for words in X_train])
X_test_vectors = np.array([get_average_word_vector(words, word2vec_model) for words in X_test])

(16363, 200)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# Train the Random Forest Classifier
classifier = RandomForestClassifier(n_estimators=100, random_state=42)
# classifier.fit(X_train_vectors, y_train)

# Predict on the test set